In [4]:
import numpy as np
import torch

pieces = [[[1, 1],[1, 1]],[[0, 2, 0],[2, 2, 2]],[[0, 3, 3],[3, 3, 0]],[[4, 4, 0],[0, 4, 4]],[[5, 5, 5, 5]],[[0, 0, 6],[6, 6, 6]],[[7, 0, 0],[7, 7, 7]]]

board_width,board_height = 7,8

def index_of(piece):
    return int(np.sum(piece)/4) - 1

def rotate(input_piece,n_times):
    i = [x[:] for x in input_piece]
    
    for _ in range(n_times):
        rows = len(i)
        cols = len(i[0])
        r = [[0]*rows for _ in range(cols)]
        
        for rs in range(rows):
            for cs in range(cols):
                #for y coordinate,downforward is positive
                r[cs][rows-rs-1] = i[rs][cs]
                # Or r[cs][rs] = i[rows-rs-1][cs]
        i = r
    return i


class Tetris:
    def __init__(self):
        self.reset()   
    def reset(self):
        self.remained = [2]*7
        self.current_step = 0        
        self.MissionAccomplished = False           
        self.current = np.array([[0]*board_width for r in range(board_height)])
    
    # Make sure new adding piece and previous pieces attach in y direction by using numpy diff()
    def attach(self,p,heights):
        # board heights difference
        pattern = np.diff(heights)
        wid = len(p[0])
        if wid == 1:#can't use numpy diff() though because shape '\'
            x = np.arange(board_width)
            y = heights
            return list(zip(y,x))
        # pd -> piece border shape height difference ie [[0, 3, 3],[3, 3, 0]]->[1,0,0]->[0-1=-1,0-0=0]->[-1,0]
        pd = np.diff(self.heights(p = p, wid = wid))
        step = board_width + 1 - wid      
        attach_x = [False] * step
        for _ in range(step):
            if (pattern[_:_+wid-1] == pd).all():
                attach_x[_] = True
        x = [i for i,z in enumerate(attach_x) if z == True]
        # check if piece[0][0] position is None
        not_empty = p[0][0]
        if not_empty:
            y = [heights[X] for X in x]
        else:
            y = [heights[X]-1 for X in x]            
        #return suitable position for adding piece attatch other pieces
        return list(zip(y,x))
        
    
    # return board empty "height" in y direction
    # or return piece "height"
    def heights(self,p, wid = board_width):
        heights = [board_height]*wid
        if wid == board_width:
            reverse = np.transpose(p[::-1])
        else:
            reverse = np.transpose(p)
        for idx,col in enumerate(reverse):
            for i,e in enumerate(col):
                if e != 0:
                    heights[idx] = i
                    break

        return np.array(heights)
    
    
    # add piece to board, only non-zero element in piece array will be added
    def board_add_pieces(self, board, remained, adding_piece, y, x, real = False):
        a = adding_piece
        idx = index_of(a)
        if real:
            self.remained[idx] = self.remained[idx] - 1
            b = board
        else:
            remained[idx] = remained[idx] - 1
            b = board.copy()
        for r in range(len(a)):
            for c in range(len(a[r])):
                if a[r][c]:
                    row = y + r
                    col = x + c
                    b[row, col] = a[r][c]
        return b, remained
    
    # check if piece over border in y direction
    def y_border(self,p,y):
        end_y = y + len(p)
        Colli = False    
        if end_y > board_height:
            Colli = True
        return Colli
    
    def pity(self,state, remained, choice):
        print(state)
        pity = False
        print(remained)
        if len(self.next_steps(state, remained)) < sum(remained):
            pity = True
        return pity 

    # find all possible choices in a single step
    def next_steps(self,board, remained_pieces, ):
        choices = {}
        heights = board_height - self.heights(board.copy())
        for i,e in enumerate(remained_pieces):
            if e > 0:
                if i == 0:
                    rot = 1
                elif i == 2 or i == 3 or i == 4:
                    rot = 2
                else:
                    rot = 4
                
                p = pieces[i]
                for r in range(rot):
                    xys = self.attach(p,heights)
                    if xys:
                        for y,x in xys:
                            colli = self.y_border(p,y)
                            if not colli:
                                choice = (i,r,x,y)
                                state, remained = self.board_add_pieces(self.current, self.remained, p, y, x)
                                if sum(remained) == 0:
                                    return -1
                                pity = self.pity(state, remained, choice)
                                if not pity:
                                    state_as_input = self.board_add_pieces(self.current, self.remained, p, y, x, real = True) 
                                    choices[choice] = state_as_input
                    p = rotate(p,n_times = 1)

        if choices == {}:
            self.gameover = True
        return choices



    def state_as_nn_input(self, heights, choice):
        diffs = self.different(choice)
        # 1.pieces remained amount  2.current status full map  3.the added piece index ,rotate time and where  4.trim
        choice = torch.FloatTensor(choice)
        input_state = torch.FloatTensor([ [1 if x else 0 for x in y] for y in self.current()])
        heights = torch.FloatTensor(heights)
        number_of_remained = torch.FloatTensor(self.remained)/2
        border_size = None
        trim = diffs
        return torch.cat((input_state,capture_row,capture_col,number_of_remained,level),0)  

    
    def different(self):
        pass


    # Made a step/choice from all possible steps/choices
    def step(self,idx,r,x,y): 
        p = pieces[idx]
        p = rotate(p,r)
        current_state,conflit = self.board_add_pieces(p,x,y,real = True)
        trim = self.trim()
        self.current_step += 1
        self.update_mountain(p,x)
        if trim:
            reward = trim * self.current_step
        else:
            reward = 0.1 * self.current_step
        self.score += reward
        return self.current_step

In [5]:
def play(tetris = Tetris()):
    choices = tetris.next_steps(tetris.current, tetris.remained)
    if choices != {}:
        actions, states = zip(*choices.items())
        idx = np.random.randint(0,len(states) - 1)
        tetris.current = states[idx]
        print(tetris.current)
        p = actions[idx][0]
        tetris.remained[p] = tetris.remained[p] - 1
    else:
        tetris.gameover = True
        print(tetris.current,'Game Over')
        print([pieces[x] for x,y in enumerate(tetris.remained) if y])
    if not (tetris.gameover):
        print('tetris.remained',tetris.remained,'tetris.remained')
        play(tetris)

In [6]:
play()

[[1 1 0 0 0 0 0]
 [1 1 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]]
[1, 2, 2, 2, 2, 2, 2]
[[0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [1 1 0 0 0 0 0]
 [1 1 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]]
[0, 2, 2, 2, 2, 2, 2]
[[0 0 2 2 2 0 0]
 [0 0 0 2 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]]
[0, 1, 2, 2, 2, 2, 2]
[[0 2 0 0 0 0 0]
 [0 2 2 0 0 0 0]
 [0 2 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]]
[0, 0, 2, 2, 2, 2, 2]
[[0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 4 0 0 0 0]
 [0 4 4 0 0 0 0]
 [0 4 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]]
[0, 0, 2, 1, 2, 2, 2]
[[0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 4 0 0 0 0]
 [0 4 4 0 0 0 0]
 [0 4 0 0 0 0 0]
 [0 0 0 0 0 0 0]]
[0, 0, 2, 0, 2, 2, 2]
[[0 0 0 5 5 5 5]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 

TypeError: object of type 'int' has no len()